In [2]:
# Wrangling
import pandas as pd
import numpy as np

# Reading
import json
import pickle

# Navigating
import os

# Import Data

## Real Estate Transactions

In [3]:
# folder navigation
print(os.path.abspath(os.curdir))

/Users/Chris/Documents/GitHub/SIADS691-Covid-Real-Estate


In [4]:
# Import 
with open('datasets/CA_data/counties_pull.txt') as project_file:    
    data = json.load(project_file)  

ca_geo_df = pd.json_normalize(data)

In [5]:
# CO06037 = LA ; CO06075 = SF
ca_geo_df[ca_geo_df.name.isin(['Los Angeles','San Francisco'])]

,geo_center_latitude,geo_center_longitude,geo_key,id,name,type
28,34.1963980000,-118.2618620000,CO06037,CO06037,Los Angeles,CO
51,37.7272390000,-123.0322290000,CO06075,CO06075,San Francisco,CO


In [36]:
# Build Dataframes from json files

# Initialize dataframe holder
ca_df_dict = {}

# Loop to build list of files for concatination
folders = ['Los Angeles','San Francisco','San Diego','San Bernardino','Sacramento','Riverside','Orange']
for folder in folders:
    print(f'Building {folder}')
    # Get directories 
    files = os.listdir("datasets/CA_data/"+folder)

    # Get json files in directory
    for county_f in files:
        with open(f'datasets/CA_data/'+folder + '/' + county_f) as project_file:
            try:
                data = json.load(project_file)
                df = pd.json_normalize(data) 

                # Concat dataframes
                if folder in ca_df_dict:
                    df2 = ca_df_dict[folder]
                    h_concat_df = pd.concat([df2, df], axis=0)
                    ca_df_dict[folder] = h_concat_df
                else:
                    ca_df_dict[folder] = df
            except:
                print(f'Failed: {project_file}')

print('Dictionary Created!')

Building Los Angeles
Building San Francisco
Building San Diego
Building San Bernardino
Building Sacramento
Building Riverside
Building Orange
Dictionary Created!


In [37]:
# Create dictionary for renaming
col_dict = {'sale.saleTransDate':'transaction_date','address.postal1':'zipcode','sale.amount.saleamt':'sale_price','lot.lotSize1':'lot_size','building.size.universalsize':'size','location.latitude':'latitude','location.longitude':'longitude','summary.propsubtype':'sub_type','summary.proptype':'type','summary.yearbuilt':'year_built','building.rooms.bathstotal':'bathrooms','building.rooms.beds':'bedrooms'}
# Create list for anti filter drop
col_drop_list = [ col for col in ca_df_dict['Orange'].columns if col not in col_dict.keys()]

In [42]:
# Clean the DataFrames
for data in ca_df_dict.keys():
    ca_df_dict[data].rename(columns = col_dict, inplace=True)
    ca_df_dict[data].drop(col_drop_list, inplace=True, axis=1, errors='ignore')
    ca_df_dict[data].reset_index(drop=True, inplace=True)
    ca_df_dict[data].dropna(subset=['transaction_date'], inplace=True) #

In [43]:
# export cleaned dictionary of dataframe
with open("ca_df_dict.pkl","wb") as ca_dump:
    pickle.dump(ca_df_dict,ca_dump)
    print('done')

done


In [10]:
total_size = 0
for bc in ca_df_dict.keys():
    total_size += ca_df_dict[bc].shape[0]
print(f'Total length is {total_size}')

Total length is 491856


In [34]:
# Read pickle back in
#ca_pickled = pd.read_pickle("datasets/CA_data/ca_df_dict.pkl")
#print('done')

done


## Income 

In [11]:
# Federal income data reported 2019
income_data = pd.read_excel("datasets/income/est19-all_income.xls", header=3)[['Postal Code', 'Name','Median Household Income']]

# Select relevant income data
income_data_CA_NY = income_data[(income_data['Postal Code'].isin(['CA'])) & income_data['Name'].str.contains('County')]

# Select relevant columns
median_income_df = income_data_CA_NY[['Postal Code', 'Name','Median Household Income']]
median_income_df.rename(columns={'Name':'County','Postal Code':'State ABV'}, inplace=True)

# Isolate each county's Area Median Income (AMI).
ca_median_income_df = median_income_df[median_income_df.County.isin(['San Francisco County','Los Angeles County','San Diego County','San Bernardino County','Sacramento County','Riverside County','Orange County'])]

ca_median_income_df

,State ABV,County,Median Household Income
210,CA,Los Angeles County,72721
221,CA,Orange County,95761
224,CA,Riverside County,72905
225,CA,Sacramento County,71891
227,CA,San Bernardino County,67398
228,CA,San Diego County,83576
229,CA,San Francisco County,121795


## Population

In [12]:
# Need to import population for Covid factorization 
# (Daily infection / Population) -> DEFAULT TO NYT
# (Daily death / Population) -> DEFAULT TO NYT

# Density for filtering on the "urban factor" & causal county comparison 
# https://worldpopulationreview.com/us-counties/states/ca
# https://worldpopulationreview.com/us-counties/states/ny
pop_density = pd.read_csv("datasets/population/ca_density_pop_by_county.csv")
pop_density['state'] = 'California'

# Rename for Title Caps
pop_density.rename(columns={'CTYNAME':'City Name','popDensity':'Pop Density','pop2021': 'Pop 2021','state':'State'}, inplace=True)

pop_density[pop_density['City Name'].isin(['San Francisco County','Los Angeles County','San Diego County','San Bernardino County','Sacramento County','Riverside County','Orange County'])]

,City Name,Pop 2021,GrowthRate,Pop Density,State
0,Los Angeles County,9969510,1.4889,13488.8513,California
1,San Diego County,3347270,7.8648,4528.8913,California
2,Orange County,3175130,5.3052,4295.9841,California
3,Riverside County,2520060,14.4660,3409.6675,California
4,San Bernardino County,2206750,8.1290,2985.7558,California
7,Sacramento County,1578680,11.0667,2135.9706,California
11,San Francisco County,883255,9.6523,1195.0533,California


## Covid-19

In [13]:
# Read in New York Times data - > SELECTED
# This represents a cumulative number
# https://github.com/nytimes/covid-19-data/blob/master/us-counties.csv
nyt_cum = pd.read_csv("datasets/covid/nyt-us-counties.csv")
nyt_cum = nyt_cum[
    (nyt_cum.county.isin(
        ['San Francisco','Los Angeles','San Diego','San Bernardino','Sacramento','Riverside','Orange'])) & (nyt_cum.state == 'California')].drop(['fips'], axis=1)
nyt_cum = nyt_cum.rename(columns={'cases':'cum_cases','deaths':'cum_deaths'})

# Next represents daily reports and averages - > SELECTED
# https://github.com/nytimes/covid-19-data/tree/master/rolling-averages
nyt_avgs = pd.read_csv("datasets/covid/nyt-rolling-us-counties.csv")
nyt_avgs = nyt_avgs[nyt_avgs.county.isin(['San Francisco','Los Angeles','San Diego','San Bernardino','Sacramento','Riverside','Orange']) & (nyt_avgs.state == 'California')].drop(['geoid'], axis=1)
nyt_avgs = nyt_avgs.rename(columns={'cases':'New Cases','cases_avg':'R7 Cases AVG','deaths':'Days Deaths','cases_avg_per_100k':'R7 Cases AVG Per 100k','deaths_avg':'R7 Deaths AVG','deaths_avg_per_100k':'R7 Deaths AVG Per 100k'})

# Merge the dataframes into one
covid = pd.merge(nyt_cum, nyt_avgs, on=['date','county','state'], how='outer')
covid.rename(columns={'date':'Date', 'county':'County', 'state':'State', 'cum_cases': 'CUM Cases', 'cum_deaths':'CUM Deaths'}, inplace=True)

In [14]:
covid.sample(2)

,Date,County,State,CUM Cases,CUM Deaths,New Cases,R7 Cases AVG,R7 Cases AVG Per 100k,Days Deaths,R7 Deaths AVG,R7 Deaths AVG Per 100k
2475,2021-02-02,Orange,California,247886.0,3162.0,851,1244.43,39.19,53,56.29,1.77
2675,2021-03-02,San Diego,California,261064.0,3317.0,376,456.57,13.68,14,14.14,0.42


In [47]:
# export covid df
with open("covid.pkl","wb") as covid_dump:
    pickle.dump(covid,covid_dump)
    print('done')

done


# Functions for Metrics, Statistics, Classification 

<em>Drilling into true affordability requires fincial data we don't have</em>

The first-time homebuyer affordability index is based upon the assumption of 70% median household income, 85% of the median house price, a 10% down payment on a 30-year fixed rate mortgage at prevailing rates plus 0.25% added per month for mortgage insurance. The assumption of 25% of gross income for mortgage repayments also applies.

The composite housing affordability index assumes a median household income, median house prices, a 20% down payment on a 30-year fixed rate mortgage at prevailing rates. The assumption is also made that the lender will not allow principal and interest costs to exceed 25% of gross income.

<em>Therefore we need to use some generic metric such as the 30% rule.</em>

A ratio of housing cost to income is the gold standard, it is known as the 30 percent rule, and it is established because of legal actions.

https://www.forbes.com/sites/rogervaldez/2020/02/07/housing-ideas-for-the-feds-find-a-new-measure-of-affordable/?sh=2e9f3c3c36f8

https://www.huduser.gov/portal/pdredge/pdr_edge_featd_article_092214.html

In [22]:
# For some reason nesting this inside summary states isn't working
def affordability(row, monthly_30):
    """
    Row wise operations for apply function
    Return row of values
    """

    # Mortgage variables
    # Cite: http://www.freddiemac.com/pmms/docs/30yr_pmmsmnth.xls
    freddie_mac_mortgage_rates = {2019:3.94,2020:3.11,2021:2.92}
    n = 30 * 12

    P = row['sale_price']
    r = freddie_mac_mortgage_rates[row['transaction_date'].year]
    row['Monthly Mortgage'] = (P * r * ((1 + r)**n)) / (((1 + r)**n) - 1)
    row['Affordability'] = 100 * ((monthly_30 - row['Monthly Mortgage']) / monthly_30 )
    return row

In [23]:
def summary_stats(df, county, ami):
    """
    Requires numpy 
    Input variable = df (dataframe) , area median income (int), county (string)

    Fixed Monthly Mortgage Repayment Calculation = P * r * (1 + r)n / [(1 + r)n – 1]
    where P = Outstanding loan amount, r = Effective monthly interest rate, 
    n = Total number of years * 12 months

    Return new unit price columns -> 'price_per_size_unit', 'price_per_bed_unit', 'price_per_lot_size_unit'
    Return class based on 'price_per_size_unit'  -> 'tier' [lower, mid , top]
    Return affordability -> 'monthly_mortgage' , 'affordability' , 'ami'
    """

    # Unit price columns
    df['Price Per Size Unit'] = np.where(df['size'] != 0, 
        round(df['sale_price'] / df['size'], 2), # True
        np.nan # False
        )
    df['Price Per Bed Unit'] = np.where(df['bedrooms'] != 0, 
        round(df['sale_price'] / df['bedrooms'], 2), # True
        np.nan # False
        )
    df['Price Per Lot Size Unit'] = np.where(df['lot_size'] != 0, 
        round(df['sale_price'] / df['lot_size'], 2), # True
        np.nan # False
        )

    # Binning into 3 equal tiers based on price
    df['Tier'] = pd.cut(
        df[
            (df['Price Per Size Unit'].notna()) & 
            (df['sub_type'].isin(['RESIDENTIAL','HOUSE']))
            ]['Price Per Size Unit'], 
        bins=3, 
        labels=['Bottom','Middle','Top'], 
        ordered=True)

    # Convert to datetime
    df['transaction_date'] = pd.to_datetime(df['transaction_date'])


    # Apply the affordability function
    # Monthly is needed for the afordability calculation
    monthly_30 = ( ami / 12 ) * .3
    df = df.apply(lambda x: affordability( row = x, monthly_30 = monthly_30), axis=1)
    

    return df


In [44]:
for county_key in sorted(ca_df_dict.keys()):
    missing_counties = list()
    try:
        ca_df_dict[county_key] = summary_stats(
                df = ca_df_dict[county_key], 
                county = county_key, 
                ami = int(median_income_df[
                        median_income_df['County'].str.contains(str(county_key))]['Median Household Income'])
        )
    except:
        missing_counties.append(county_key)
        print(f"County income data mising for {county_key}. Continuing...")
        continue

In [45]:
# export cleaned dictionary of dataframe
with open("ca_df_dict_w_stats.pkl","wb") as ca_dump:
    pickle.dump(ca_df_dict,ca_dump)
    print('done')

done


In [ ]:
"""
ca_df_dict['Orange'] = summary_stats(
    df= ca_df_dict['Orange'], 
    county = 'Orange', 
    ami = int(median_income_df[median_income_df['County'].str.contains('Orange')]['Median Household Income'])
    )
    
ca_df_dict['San Diego'] = summary_stats(
    df= ca_df_dict['San Diego'], 
    county = 'San Diego', 
    ami = int(median_income_df[median_income_df['County'].str.contains('San Diego')]['Median Household Income'])
    )

ca_df_dict['San Francisco'] = summary_stats(
    df= ca_df_dict['San Francisco'], 
    county = 'San Francisco', 
    ami = int(median_income_df[median_income_df['County'].str.contains('San Francisco')]['Median Household Income'])
    )

ca_df_dict['Los Angeles'] = summary_stats(
    df= ca_df_dict['Los Angeles'], 
    county = 'Los Angeles', 
    ami = int(median_income_df[median_income_df['County'].str.contains('Los Angeles')]['Median Household Income'])
    )

ca_df_dict['Scremento'] = summary_stats(
    df= ca_df_dict['Scremento'], 
    county = 'Scremento', 
    ami = int(median_income_df[median_income_df['County'].str.contains('Scremento')]['Median Household Income'])
    )
    
ca_df_dict['San Bernardino'] = summary_stats(
    df= ca_df_dict['San Bernardino'], 
    county = 'San Bernardino', 
    ami = int(median_income_df[median_income_df['County'].str.contains('San Bernardino')]['Median Household Income'])
    )

ca_df_dict['Riverside'] = summary_stats(
    df= ca_df_dict['Riverside'], 
    county = 'Riverside', 
    ami = int(median_income_df[median_income_df['County'].str.contains('Riverside')]['Median Household Income'])
    )

ca_df_dict['Riverside'][ca_df_dict['Riverside']['transaction_date'].isna()]
ca_df_dict['Los Angeles'][ca_df_dict['Los Angeles']['transaction_date'].isna()]
ca_df_dict['San Diego'][ca_df_dict['San Diego']['transaction_date'].isna()]

"""